citc: rl

blaze-bin/experimental/users/ppiech/rl/notebook.par > /dev/null 2>&1 & disown

In [1]:
from __future__ import absolute_import, division, print_function

from absl import logging
import sys
import time
import os
import subprocess
import base64
import imageio
import ffmpeg
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import random

import tensorflow as tf

from spaceship_env import SpaceshipEnv
from spaceship_agent import Agent

import video_util


2023-12-14 16:42:48.423965: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 16:42:48.457352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 16:42:48.457995: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 16:42:49.002268: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


pygame 2.1.3 (SDL 2.0.22, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
num_iterations = 40000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"}
collect_steps_per_iteration = 10 # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 5000  # @param {type:"integer"}

log_interval = 1000  # @param {type:"integer"}
train_checkpoint_interval=10000 # @param {type:"integer"}
policy_checkpoint_interval=5000 # @param {type:"integer"}
rb_checkpoint_interval=2000 # @param {type:"integer"}
keep_rb_checkpoint=False # @param {type:"boolean"}
train_sequence_length=1 # @param {type:"integer"}
summary_interval=1000 # @param {type:"integer"}
summaries_flush_secs=10 # @param {type:"integer"}

In [3]:
root_dir = os.path.expanduser('log')
train_dir = os.path.join(root_dir, 'train')
eval_dir = os.path.join(root_dir, 'eval')
saved_model_dir = os.path.join(root_dir, 'policy_saved_model')
if not tf.io.gfile.exists(saved_model_dir):
  tf.io.gfile.makedirs(saved_model_dir)

In [4]:
env = SpaceshipEnv()

def create_random_video(filename, fps=30):
  filename = filename + "/starship_env_random.mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    env.reset()
    video.append_data(env.render())
    for _ in range(100):
      env.step(random.randint(-1, 1))
      video.append_data(env.render())
  return video_util.embed_mp4(filename)

create_random_video("log/eval")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1400, 1000) to (1408, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5eafd80] Warning: data is not aligned! This can lead to a speed loss


In [8]:
def train_model(agent, env, num_episodes, file=None):
  if file:
    agent.load(file, env)
        
  render = True

  scores, episodes, avg_scores, obj = [], [], [], []
  goal = 200
  f = 0
  steps = 0
  train = False

  for i in range(num_episodes):
    if i % 300 == 0 and i != 0:
      agent.save(f)
      f += 1
        
    done = False
    score = 0.0
    state, _ = env.reset()
    while not done:
      action = agent.policy(state)
      new_state, reward, terminated, truncated, _ = env.step(action)
      done = terminated
      score += reward
      agent.store_tuple(state, action, reward, new_state, done)
      state = new_state
      if steps % 10 and train == 0:
          agent.train()
      steps += 1
    # if self.buffer.counter > self.buffer.size - 100:
    #         train = True
    scores.append(score)
    obj.append(goal)
    episodes.append(i)
    avg_score = np.mean(scores[-100:])
    avg_scores.append(avg_score)
    
    print("Episode {0}/{1}, Score: {2} ({3}), AVG Score: {4}".format(i, num_episodes, score, agent.epsilon,
                                                                       avg_score))
  # if avg_score >= 200.0 and score >= 250:
  agent.save(f)
  f += 1

In [9]:
num_episodes = 3
graph = True

file = os.path.join(saved_model_dir, 'space_model_best')

# EPSILON LOWERED FOR NOW

dqn_agent = Agent(lr=0.00075, discount_factor=0.99, num_actions=3, epsilon=0.03, batch_size=5000, input_dims=7)

train_model(dqn_agent, env, num_episodes, file=None)


Episode 0/3, Score: -100004.20000000007 (0.03), AVG Score: -100004.20000000007
Episode 1/3, Score: -100198.10000000267 (0.03), AVG Score: -100101.15000000136
Episode 2/3, Score: -100040.70000000056 (0.03), AVG Score: -100081.00000000109
f: 0
INFO:tensorflow:Assets written to: saved_networks/space_model0/assets


INFO:tensorflow:Assets written to: saved_networks/space_model0/assets


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = 'saved_networks/dqn_model0/net_weights0.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

## Stop here

In [ ]:
train_env = SpaceshipEnv()
eval_env = SpaceshipEnv()

In [ ]:
#
# Policy
#
fc_layer_params = (256, 256)
action_tensor_spec = tensor_spec.from_spec(train_py_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [ ]:
#
# Agent
#
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0, dtype=tf.int64)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

policy_checkpointer = common.Checkpointer(
    ckpt_dir=os.path.join(train_dir, 'policy'),
    max_to_keep=None,
    policy=agent.policy,
    global_step=agent.train_step_counter)

saved_model = policy_saver.PolicySaver(
    agent.policy)

def save_policy(global_step_value):
    """Saves policy using both checkpoint saver and saved model."""
    policy_checkpointer.save(global_step=global_step_value)
    saved_model_path = os.path.join(
        saved_model_dir, 'policy_' + ('%d' % global_step_value).zfill(8))
    saved_model.save(saved_model_path)

eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
#
# Replay buffer
#

table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

# table = reverb.Table(
#     table_name,
#     max_size=replay_buffer_max_length,
#     sampler=reverb.selectors.Uniform(),
#     remover=reverb.selectors.Fifo(),
#     rate_limiter=reverb.rate_limiters.MinSize(1),
#     signature=replay_buffer_signature)

# reverb_server = reverb.Server([table])

# replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
#     agent.collect_data_spec,
#     table_name=table_name,
#     sequence_length=2,
#     local_server=reverb_server)

# rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
#   replay_buffer.py_client,  #not py_client
#   table_name,
#   sequence_length=2)

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec, 
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

print('RB capacity: %i' % replay_buffer.capacity)
rb_observer = replay_buffer.add_batch

rb_ckpt_dir = os.path.join(train_dir, 'replay_buffer')
rb_checkpointer = common.Checkpointer(
    ckpt_dir=rb_ckpt_dir, max_to_keep=1, replay_buffer=replay_buffer)


In [ ]:
#
# Train Metrics
#

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(batch_size=train_env.batch_size),
    tf_metrics.AverageEpisodeLengthMetric(batch_size=train_env.batch_size),
    tf_metrics.ChosenActionHistogram(),
]

train_checkpointer = common.Checkpointer(
    ckpt_dir=train_dir,
    max_to_keep=1,
    agent=agent,
    global_step=agent.train_step_counter,
    metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))


In [ ]:
#
# Driver
#
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec(), 
                                                info_spec=agent.collect_policy.info_spec)

random_policy = agent.collect_policy

agent_observers = [replay_buffer.add_batch]

DynamicStepDriver(
    train_env, agent.collect_policy, [rb_observer], num_steps=1000).run()
# driver.num_steps = 1000
# driver.run()

# py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       random_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=initial_collect_steps).run(train_py_env.reset())

# # Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

iterator = iter(dataset)

In [ ]:
#
# Eval Metrics
#

eval_summary_writer = tf.compat.v2.summary.create_file_writer(
    eval_dir, 
    flush_millis=summaries_flush_secs * 1000
)

eval_metrics = [
    tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
    tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes),
]

eval_policy =  greedy_policy.GreedyPolicy(agent.policy)
 
metric_utils.eager_compute(
    eval_metrics,
    eval_env,
    eval_policy,
    num_episodes=num_eval_episodes,
    summary_writer=eval_summary_writer,
    train_step=0
)

metric_utils.log_metrics(eval_metrics)

In [ ]:
#
# Train Loop
#

try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Reset the environment.
time_step = train_env.reset()

collect_driver = DynamicStepDriver(
    train_env, agent.collect_policy, [rb_observer], 
    num_steps=collect_steps_per_iteration)

timed_at_step = agent.train_step_counter.numpy()
time_acc = 0

for _ in range(num_iterations):
  start_time = time.time()

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step_tf = agent.train_step_counter
  step = step_tf.numpy()

  time_acc += time.time() - start_time

  for train_metric in train_metrics:
    train_metric.tf_summaries(
        train_step=agent.train_step_counter, step_metrics=train_metrics[:2])

  if step % train_checkpoint_interval == 0:
    train_checkpointer.save(global_step=step)

  if step % policy_checkpoint_interval == 0:
    save_policy(step)

  if step % rb_checkpoint_interval == 0:
    rb_checkpointer.save(global_step=step)

  if step % log_interval == 0:
    print('step = %d, loss = %f' % (step, train_loss.numpy()))
    steps_per_sec = (step- timed_at_step) * collect_steps_per_iteration / time_acc
    print('%.3f steps/sec' % steps_per_sec)
    tf.compat.v2.summary.scalar(
        name='env_steps_per_sec', data=steps_per_sec, step=step_tf)
    timed_at_step = step
    time_acc = 0

  if step % eval_interval == 0:
    metric_utils.eager_compute(
      eval_metrics,
      eval_env,
      eval_policy,
      num_episodes=num_eval_episodes,
      summary_writer=eval_summary_writer,
      train_step=step
    )

    metric_utils.log_metrics(eval_metrics)


In [ ]:
metric_utils.eager_compute(
      eval_metrics,
      eval_env,
      eval_policy,
      num_episodes=num_eval_episodes,
      summary_writer=eval_summary_writer,
      train_step=step
    )

print (eval_metrics)

In [ ]:
train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(batch_size=train_env.batch_size),
    tf_metrics.AverageEpisodeLengthMetric(batch_size=train_env.batch_size),
    tf_metrics.ChosenActionHistogram(),
]

train_checkpointer = common.Checkpointer(
    ckpt_dir=train_dir,
    max_to_keep=1,
    agent=agent,
    global_step=agent.train_step_counter,
    metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))



In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")